In [9]:
import numpy as np
 
class Particle:
    def __init__(self, position, velocity, mass, particle_type):
        """
        Initializes a particle with position, velocity, mass, and type.
        """
        self.position = np.array(position, dtype=float)  # Position vector [x, y, z]
        self.velocity = np.array(velocity, dtype=float)  # Velocity vector [vx, vy, vz]
        self.mass = mass                                 # Mass of the particle
        self.type = particle_type                        # Particle type (e.g., "proton", "electron")
        self.energy = self.calculate_kinetic_energy()    # Initial energy calculated
 
    def update_position(self, dt):
        """
        Updates the particle’s position based on its velocity and a given time step dt.
        """
        self.position += self.velocity * dt
 
    def calculate_kinetic_energy(self):
        """
        Calculates the kinetic energy of the particle.
        KE = 0.5 * mass * velocity^2
        """
        velocity_magnitude = np.linalg.norm(self.velocity)
        kinetic_energy = 0.5 * self.mass * velocity_magnitude ** 2
        return kinetic_energy
 
    def detect_decay(self):
        """
        Detects if the particle should decay. This can be based on energy or random probability.
        """
        # Basic example: particles with high energy have a chance to decay
        decay_probability = min(1, self.energy / 1000)  # Arbitrary threshold for demonstration
        if np.random.random() < decay_probability:
            print(f"{self.type} particle decayed!")
            return self.decay()
        return None
 
    def decay(self):
        """
        Decay mechanism where the particle may transform into other particles.
        Returns a list of new particles or an empty list if no decay products.
        """
        decay_products = []
        
        # For demonstration, let's assume that a proton decays into two lighter particles
        if self.type == "proton":
            decay_products.append(Particle(self.position, self.velocity / 2, self.mass / 2, "neutron"))
            decay_products.append(Particle(self.position, -self.velocity / 2, self.mass / 2, "positron"))
 
        # Decay can be expanded based on type and probabilities
        return decay_products
 
    def display_info(self):
        """
        Prints particle information: type, mass, position, velocity, and energy.
        """
        print(f"Particle Type: {self.type}")
        print(f"Mass: {self.mass} kg")
        print(f"Position: {self.position}")
        print(f"Velocity: {self.velocity}")
        print(f"Kinetic Energy: {self.energy} J")

In [11]:
import numpy as np

# Constants
k_e = 8.9875517873681764e9  # Coulomb's constant in N m²/C²
fusion_energy_threshold = 1e12  # Example threshold for fusion in joules

class Collider:
    def __init__(self, particles):
        self.particles = particles

    def detect_collisions(self):
        """Detect collisions based on proximity between particles."""
        collision_threshold = 1e-10  # Distance threshold for collision detection
        for i, p1 in enumerate(self.particles):
            for j, p2 in enumerate(self.particles[i+1:], start=i+1):
                distance = np.linalg.norm(p1.position - p2.position)
                if distance < collision_threshold:
                    self.handle_collision(p1, p2)

    def handle_collision(self, p1, p2):
        """Handle a collision between two particles, with conservation laws and potential fusion or transformation."""
        # Initial energy and momentum for conservation checks
        total_initial_momentum = p1.mass * p1.velocity + p2.mass * p2.velocity
        total_initial_energy = p1.total_energy() + p2.total_energy()
        total_charge = p1.charge + p2.charge  # Ensure charge conservation

        # Check if fusion occurs based on total energy
        if total_initial_energy > fusion_energy_threshold:
            self.perform_fusion(p1, p2, total_initial_momentum, total_charge)
        else:
            # If no fusion, perform a standard collision with momentum and energy conservation
            self.apply_momentum_conservation(p1, p2)

            # Implement stochastic transformation (Monte Carlo)
            self.stochastic_transformation(p1, p2)

    def apply_momentum_conservation(self, p1, p2):
        """Apply conservation of momentum and energy during elastic collision."""
        # Calculate final velocities using conservation of momentum and energy
        if p1.mass != p2.mass:
            v1_final = ((p1.mass - p2.mass) / (p1.mass + p2.mass)) * p1.velocity + (2 * p2.mass / (p1.mass + p2.mass)) * p2.velocity
            v2_final = ((p2.mass - p1.mass) / (p1.mass + p2.mass)) * p2.velocity + (2 * p1.mass / (p1.mass + p2.mass)) * p1.velocity
            p1.velocity = v1_final
            p2.velocity = v2_final
        else:
            # If masses are equal, swap velocities
            p1.velocity, p2.velocity = p2.velocity, p1.velocity

    def stochastic_transformation(self, p1, p2):
        """Randomly transform particles post-collision based on a probability."""
        transformation_probability = 0.3  # Example probability for transformation

        if np.random.rand() < transformation_probability:
            # Example of splitting a particle into two new particles
            new_mass = p1.mass * 0.5
            p1.mass *= 0.5
            p2.mass *= 0.5
            p1.type = "DecayProduct1"
            p2.type = "DecayProduct2"
            print(f"Particles transformed into new types {p1.type} and {p2.type} with stochastic transformation.")

    def perform_fusion(self, p1, p2, total_initial_momentum, total_charge):
        """Fuse two particles into a single new particle with combined mass and charge."""
        combined_mass = p1.mass + p2.mass
        additional_mass = ((p1.total_energy() + p2.total_energy()) - (combined_mass * c**2)) / c**2
        new_mass = combined_mass + additional_mass
        new_velocity = total_initial_momentum / new_mass
        fusion_particle = Particle(position=p1.position, velocity=new_velocity, mass=new_mass, charge=total_charge, p_type="FusionParticle")
        self.particles.append(fusion_particle)
        print(f"Fusion created a new particle with mass {new_mass} and charge {total_charge}.")

    def calculate_electric_force(self, p1, p2):
        """Calculate the electric force between two charged particles using Coulomb's law."""
        distance_vector = p2.position - p1.position
        distance = np.linalg.norm(distance_vector)
        if distance == 0:
            return np.zeros(3)  # Avoid division by zero if particles overlap
        force_magnitude = k_e * (p1.charge * p2.charge) / distance**2
        force_direction = distance_vector / distance
        return force_magnitude * force_direction

    def apply_forces(self):
        """Apply electric forces between all particle pairs and update velocities accordingly."""
        time_step = 0.01
        for i, p1 in enumerate(self.particles):
            for p2 in self.particles[i+1:]:
                electric_force = self.calculate_electric_force(p1, p2)
                p1_acceleration = electric_force / p1.mass
                p2_acceleration = -electric_force / p2.mass
                p1.velocity += p1_acceleration * time_step
                p2.velocity += p2_acceleration * time_step

In [46]:
import numpy as np
import tkinter as tk
from tkinter import messagebox
import threading
import random

class Simulation:
    def __init__(self, num_particles=10, speed=1000000, time_step=0.001, max_time=10, particle_type="proton", energy_ev=1e13):
        self.num_particles = num_particles
        self.speed = speed
        self.time_step = time_step
        self.max_time = max_time
        self.particle_type = particle_type
        self.energy_ev = energy_ev
        self.energy_joules = self.convert_ev_to_joules(energy_ev)
        self.particles = []  
        self.time = 0
        self.running = False
        self.simulation_thread = None
        self.create_particles()

    def convert_ev_to_joules(self, energy_ev):
        """ Converts energy in electron volts (eV) to joules (J). """
        return energy_ev * 1.60218e-19

    def create_particles(self):
        """ Creates particles based on the user-defined number, type, speed, and energy. """
        self.particles = []
        for _ in range(self.num_particles):
            position = np.random.rand(3) * 100  # Random positions
            velocity = np.random.rand(3) * self.speed  # Random velocity
            mass = random.uniform(1e-27, 1e-24)  # Random mass for particles
            self.particles.append(Particle(position, velocity, mass, self.particle_type))

    def update(self):
        """ Updates the simulation, checking for collisions, particle movement, and decay events. """
        if self.running:
            self.time += self.time_step
            collider = Collider(self.particles)
            collider.detect_collisions()
            for particle in self.particles:
                particle.update_position(self.time_step)
                decay_products = particle.detect_decay()
                if decay_products:
                    self.particles.extend(decay_products)

    def start(self):
        """ Starts the simulation in a new thread. """
        self.running = True
        self.simulation_thread = threading.Thread(target=self.run_simulation)
        self.simulation_thread.start()

    def pause(self):
        """ Pauses the simulation. """
        self.running = False

    def reset(self):
        """ Resets the simulation to its initial state. """
        self.time = 0
        self.running = False
        self.create_particles()

    def resume(self):
        """ Resumes the simulation from where it was paused. """
        if not self.running:
            self.running = True
            self.simulation_thread = threading.Thread(target=self.run_simulation)
            self.simulation_thread.start()

    def run_simulation(self):
        """ Runs the simulation loop until the maximum time is reached or the simulation is stopped. """
        while self.running and self.time < self.max_time:
            self.update()


# GUI for controlling the simulation
class SimulationGUI:
    def __init__(self, root, simulation):
        self.root = root
        self.simulation = simulation
        self.root.title("Particle Collider Simulation")
        self.root.geometry("400x500")  

        # Main frame for input controls
        self.form_frame = tk.Frame(root)
        self.form_frame.pack(padx=10, pady=10)

        # Number of particles input field
        self.num_particles_label = tk.Label(self.form_frame, text="Number of Particles:")
        self.num_particles_label.grid(row=0, column=0, pady=5)
        self.num_particles_entry = tk.Entry(self.form_frame, width=30)
        self.num_particles_entry.insert(0, "10")  # Default number of colliding particles 
        self.num_particles_entry.grid(row=0, column=1, pady=5)

        # Particle speed input field
        self.speed_label = tk.Label(self.form_frame, text="Particle Speed (m/s):")
        self.speed_label.grid(row=1, column=0, pady=5)
        self.speed_entry = tk.Entry(self.form_frame, width=30)
        self.speed_entry.insert(0, "1000000")  # Default speed
        self.speed_entry.grid(row=1, column=1, pady=5)

        # Time step input field
        self.time_step_label = tk.Label(self.form_frame, text="Time Step (seconds):")
        self.time_step_label.grid(row=2, column=0, pady=5)
        self.time_step_entry = tk.Entry(self.form_frame, width=30)
        self.time_step_entry.insert(0, "0.001")  # Default time step
        self.time_step_entry.grid(row=2, column=1, pady=5)

        # Particle type dropdown menu
        self.particle_type_label = tk.Label(self.form_frame, text="Select Particle Type:")
        self.particle_type_label.grid(row=3, column=0, pady=5)
        self.particle_type_var = tk.StringVar()
        self.particle_type_var.set("proton")  # Default particle type
        self.particle_type_menu = tk.OptionMenu(self.form_frame, self.particle_type_var, "proton", "electron", "neutron")
        self.particle_type_menu.config(width=20)
        self.particle_type_menu.grid(row=3, column=1, pady=5)

        # Energy of colliding particles input field
        self.energy_label = tk.Label(self.form_frame, text="Energy of Colliding Particles (eV):")
        self.energy_label.grid(row=4, column=0, pady=5)
        self.energy_entry = tk.Entry(self.form_frame, width=30)
        self.energy_entry.insert(0, "1e13")  # Default energy
        self.energy_entry.grid(row=4, column=1, pady=5)

        # Status Label (Simulation Status)
        self.status_label = tk.Label(self.form_frame, text="Status: Not Started", width=40, height=2, relief="sunken")
        self.status_label.grid(row=5, column=0, columnspan=2, pady=10)

        # Control buttons
        self.start_button = tk.Button(self.form_frame, text="Start", width=20, command=self.start_simulation)
        self.start_button.grid(row=6, column=0, columnspan=2, pady=10)

        self.pause_button = tk.Button(self.form_frame, text="Pause", width=20, command=self.pause_simulation)
        self.pause_button.grid(row=7, column=0, columnspan=2, pady=5)

        self.reset_button = tk.Button(self.form_frame, text="Reset", width=20, command=self.reset_simulation)
        self.reset_button.grid(row=8, column=0, columnspan=2, pady=5)

        self.resume_button = tk.Button(self.form_frame, text="Resume", width=20, command=self.resume_simulation)
        self.resume_button.grid(row=9, column=0, columnspan=2, pady=5)

    def start_simulation(self):
        """ Starts the simulation with the user inputs. """
        try:
            num_particles = int(self.num_particles_entry.get())
            speed = float(self.speed_entry.get())
            time_step = float(self.time_step_entry.get())
            particle_type = self.particle_type_var.get()
            energy_ev = float(self.energy_entry.get())
            self.simulation.num_particles = num_particles
            self.simulation.speed = speed
            self.simulation.time_step = time_step
            self.simulation.particle_type = particle_type
            self.simulation.energy_ev = energy_ev
            self.simulation.energy_joules = self.simulation.convert_ev_to_joules(energy_ev)
            self.simulation.create_particles()
            self.simulation.start()

            # Update status label
            self.status_label.config(text="Simulation Status: Running")
        except ValueError:
            messagebox.showerror("Input Error", "Please enter valid numerical values.")

    def pause_simulation(self):
        """ Pauses the simulation. """
        self.simulation.pause()
        self.status_label.config(text="Simulation Status: Paused")

    def reset_simulation(self):
        """ Resets the simulation to its initial state. """
        self.simulation.reset()
        self.status_label.config(text="Simulation Status: Not Started")

    def resume_simulation(self):
        """ Resumes the simulation from where it was paused. """
        self.simulation.resume()
        self.status_label.config(text="Simulation Status: Running")


# Main function to set up and run the GUI
def main():
    root = tk.Tk()
    simulation = Simulation()
    gui = SimulationGUI(root, simulation)
    root.mainloop()

if __name__ == "__main__":
    main()
